In [ ]:
import cv2
import torch
import ultralytics

ultralytics.checks()

In [ ]:
from ultralytics import YOLO
from ultralytics.trackers.byte_tracker import BYTETracker
from types import SimpleNamespace

# 1. Load YOLO11 detector
model = YOLO("yolo11n.pt")

# 2. Create tracker with your desired thresholds
args = SimpleNamespace(
    track_buffer=30,
    track_high_thresh=0.6,
    track_low_thresh=0.1,
    match_thresh=0.8,
    new_track_thresh=0.6,
    fuse_score=False
)
tracker = BYTETracker(args, frame_rate=30)

# 3. For each frame:
results = model.track(source="Homebrew-video/Low-quality/IMG_5354.MP4", imgsz=640)[0]  # results for frame
# Advance tracker
_ = tracker.update(results.boxes)
# Extract Kalman states
kalman_states = {
    track.track_id: track.mean.copy()
    for track in tracker.tracked_stracks
    if track.is_activated
}

In [ ]:
from ultralytics import YOLO
from ultralytics.trackers.byte_tracker import BYTETracker
from types import SimpleNamespace

# 1. Load YOLO11 detector
model = YOLO("yolo11x.pt")

# 2. Create tracker with your desired thresholds
args = SimpleNamespace(
    track_buffer=30,
    track_high_thresh=0.25,
    track_low_thresh=0.1,
    match_thresh=0.8,
    new_track_thresh=0.25,
    fuse_score=True
)
tracker = BYTETracker(args, frame_rate=30)

# 3. For each frame:
results = model.track(source="Homebrew-video/Low-quality/IMG_5354.MP4", imgsz=1280, classes=[0], stream=True, save=True)  # results for frame
for r in results:
    print(r.boxes.id)
    # Advance tracker
    _ = tracker.update(r.boxes)
    # Extract Kalman states
    kalman_states = {
        track.track_id: track.mean.copy()
        for track in tracker.tracked_stracks
        if track.is_activated
    }
    print(kalman_states)

In [ ]:
from ultralytics import YOLO
from ultralytics.trackers.bot_sort import BOTSORT
from types import SimpleNamespace

# 1. Load YOLO11 detector
model = YOLO("yolo11x.pt")

# 2. Create tracker with your desired thresholds
args = SimpleNamespace(
    track_buffer=30,
    track_high_thresh=0.25,
    track_low_thresh=0.1,
    match_thresh=0.8,
    new_track_thresh=0.25,
    fuse_score=True,
    # BoT-SORT settings
    gmc_method="sparseOptFlow", # method of global motion compensation
    # ReID model related thresh (not supported yet)
    proximity_thresh=0.5,
    appearance_thresh=0.25,
    with_reid=False
)
tracker = BOTSORT(args, frame_rate=30)

# 3. For each frame:
results = model.predict(source="Homebrew-video/Low-quality/IMG_5354.MP4", imgsz=1280, classes=[0], stream=True, save=False)  # results for frame
for i,r in enumerate(results):
    # Advance tracker
    _ = tracker.update(r.boxes.cpu(),r.orig_img)
    # Extract Kalman states
    kalman_states = {
        track.track_id: track.mean.copy()
        for track in tracker.tracked_stracks
        if track.is_activated
    }
    #print(kalman_states,"\n",len(kalman_states))
    if i==3: break